In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('train_data.csv',
                header=0, sep=',',
                parse_dates=['date', 'release_date', 'update_date'],
                infer_datetime_format=True, encoding = 'latin1')

In [3]:
df2 = pd.read_csv('test_data.csv',
                header=0, sep=',',
                parse_dates=['date', 'release_date', 'update_date'],
                infer_datetime_format=True, encoding = 'latin1')

In [8]:
data = df1.append(df2, ignore_index = True)[['asin','categories', 'description', 'review_id','review_text','reviewer','star_rating']]
data = data.rename(columns = {'asin': 'app_id'})

In [61]:
train.append(miss_star_rating).tail(501757-436680)

,review_text,star_rating
670839,Best game ever,NaN
670840,this is a very fun and challenging game we ne...,NaN
670843,This game really keeps you thinking. Lets you ...,NaN
670845,I got this for the FAOTD and thought my 3 year...,NaN
670846,Awesome game! I have scribblenauts for DSi and...,NaN
670847,My son is having a great time with this game. ...,NaN
670850,Loved this game and now havent been able to pl...,NaN
670851,The interface is amazingly simple and intuitiv...,NaN
670852,I love this game it is so addictive and I can ...,NaN
670853,"This game is really really fun, though I would...",NaN


In [9]:
data = data.dropna(subset = ['categories'])
data = data[data['categories'].str.contains('game', case = False)]

In [10]:
data = data.dropna(subset = ['review_text'])

In [25]:
miss_star_rating = data[~np.isfinite(data['star_rating'])]
miss_star_rating = miss_star_rating[['review_text', 'star_rating']]
train = data[np.isfinite(data['star_rating'])]

In [28]:
train = train[['review_text', 'star_rating']]

In [37]:
text = train['review_text']

ls = text.values.tolist()
import string
exclude = set(string.punctuation)
for i in range(len(ls)):
    ls[i] = ''.join(ch for ch in ls[i] if ch not in exclude)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 0.005,max_df=0.8)
dtm = vectorizer.fit_transform(ls)
terms = vectorizer.get_feature_names()

In [ ]:
X_train = train['star_rating']

In [16]:
Y = train['star_rating']

In [43]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dtm, Y, test_size=0.5, random_state=0)

X_train = pd.DataFrame(X_train.todense(),columns = vectorizer.get_feature_names())

X_test = pd.DataFrame(X_test.todense(),columns = vectorizer.get_feature_names())

ValueError: Found arrays with inconsistent numbers of samples: [436680 501757]

In [18]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred1 = gnb.predict(X_test)
nb_avg_error = ((abs(y_test - y_pred1)).sum())/len(y_pred1)
print('Naive Bayes Misclassified samples: %d'% (y_test != y_pred1).sum())
print('Naive Bayes average error: %.6f'% nb_avg_error)
from sklearn.metrics import accuracy_score
print('Naive Bayes Accuracy: %.2f'% accuracy_score(y_test, y_pred1))

Naive Bayes Misclassified samples: 109169
Naive Bayes average error: 0.922328
Naive Bayes Accuracy: 0.50


In [22]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=100, 
                                random_state=1,
                                n_jobs=2)
forest.fit(X_train, y_train)

y_pred_forest = forest.predict(X_test)
rf_avg_error = ((abs(y_test - y_pred_forest)).sum())/len(y_pred_forest)
print('RF Misclassified samples: %d'% (y_test != y_pred_forest).sum())
print('RF average error: %.6f'% rf_avg_error)
from sklearn.metrics import accuracy_score

print('RF Accuracy: %.2f'% accuracy_score(y_test, y_pred_forest))

RF Misclassified samples: 81361
RF average error: 0.684648
RF Accuracy: 0.63


In [20]:
from sklearn import linear_model
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

y_train_pred = regr.predict(X_train)
y_test_pred = regr.predict(X_test)
regr_avg_error = ((abs(y_test - y_test_pred)).sum())/len(y_test_pred)
print('Linear regression average error: %.6f'% regr_avg_error)
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

print('MSE train: %.3f, test: %.3f' %(mean_squared_error(y_train, y_train_pred),
                                      mean_squared_error(y_test, y_test_pred)))
 
print('R^2 train: %.3f, test: %.3f' %(r2_score(y_train, y_train_pred),
                                      r2_score(y_test, y_test_pred)))

Linear regression average error: 0.745066
MSE train: 0.946, test: 0.955
R^2 train: 0.502, test: 0.497


In [21]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1000.0, random_state=0)
lr.fit(X_train, y_train)
        
y_pred = lr.predict(X_test)
print('Misclassified samples: %d' % (y_test != y_pred).sum())

from sklearn.metrics import accuracy_score
lr_avg_error = ((abs(y_test - y_pred)).sum())/len(y_pred)
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))    
print('Logistic regression average error: %.6f'% lr_avg_error)

Misclassified samples: 75688
Accuracy: 0.65
Logistic regression average error: 0.577489


In [57]:
text = train.append(miss_star_rating)['review_text']

In [58]:
ls = text.values.tolist()
import string
exclude = set(string.punctuation)
for i in range(len(ls)):
    ls[i] = ''.join(ch for ch in ls[i] if ch not in exclude)

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 0.002,max_df=0.8)
dtm = vectorizer.fit_transform(ls)
terms = vectorizer.get_feature_names()

In [62]:
X = pd.DataFrame(dtm.todense(),columns = vectorizer.get_feature_names())

In [65]:
X_known = X.head(436680)
Y_known = train.append(miss_star_rating)['star_rating'].head(436680)
X_unknown = X.tail(501757-436680)
Y_unknown = train.append(miss_star_rating)['star_rating'].tail(501757-436680)

In [66]:
lr.fit(X_known, Y_known)

LogisticRegression(C=1000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=0,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [67]:
y_pred = lr.predict(X_unknown)

In [68]:
miss_star_rating['star_rating'] = y_pred

In [69]:
miss_star_rating

,review_text,star_rating
670839,Best game ever,5.0
670840,this is a very fun and challenging game we ne...,5.0
670843,This game really keeps you thinking. Lets you ...,5.0
670845,I got this for the FAOTD and thought my 3 year...,5.0
670846,Awesome game! I have scribblenauts for DSi and...,5.0
670847,My son is having a great time with this game. ...,5.0
670850,Loved this game and now havent been able to pl...,5.0
670851,The interface is amazingly simple and intuitiv...,5.0
670852,I love this game it is so addictive and I can ...,5.0
670853,"This game is really really fun, though I would...",5.0


In [ ]:
data.to_csv('data.csv', index = False)